In [2]:
from kafka import KafkaProducer
import json
import time
import random

producer = KafkaProducer(
    bootstrap_servers="localhost:9092",
    value_serializer=lambda v: json.dumps(v).encode("utf-8"),
)

flights_info = {
    "FL1000": {
        "origin": (40.6413, -73.7781),
        "destination": (51.4700, -0.4543),
    },  # JFK -> LHR
    "FL1001": {
        "origin": (34.0522, -118.2437),
        "destination": (48.8566, 2.3522),
    },  # LAX -> CDG
    "FL1002": {
        "origin": (25.2048, 55.2708),
        "destination": (1.3521, 103.8198),
    },  # DXB -> SIN
    "FL1003": {
        "origin": (35.6895, 139.6917),
        "destination": (37.7749, -122.4194),
    },  # Tokyo -> San Francisco
    "FL1004": {
        "origin": (55.7558, 37.6173),
        "destination": (41.9028, 12.4964),
    },  # Moscow -> Rome
    "FL1005": {
        "origin": (39.9042, 116.4074),
        "destination": (19.0760, 72.8777),
    },  # Beijing -> Mumbai
    "FL1006": {
        "origin": (52.5200, 13.4050),
        "destination": (40.7128, -74.0060),
    },  # Berlin -> New York
    "FL1007": {
        "origin": (51.5074, -0.1278),
        "destination": (35.6895, 139.6917),
    },  # London -> Tokyo
}


statuses_progression = ["On Time", "Delayed", "Boarding", "In Air", "Arrived"]

flights_state = {}
for flight_id, info in flights_info.items():
    flights_state[flight_id] = {
        "status": "On Time",
        "departure_time": int(time.time()),
        "arrival_time": int(time.time()) + random.randint(3600, 7200),
    }

while True:
    flight_id = random.choice(list(flights_info.keys()))
    state = flights_state[flight_id]

    current_status = state["status"]
    if current_status != "Arrived":
        next_status = random.choice(statuses_progression)
    else:
        next_status = "Arrived"

    state["status"] = next_status
    state["arrival_time"] += random.randint(-600, 900)

    flight = {
        "flight_id": flight_id,
        "origin": flights_info[flight_id]["origin"],
        "destination": flights_info[flight_id]["destination"],
        "status": state["status"],
        "departure_time": state["departure_time"],
        "arrival_time": state["arrival_time"],
    }

    producer.send("flights", flight)
    print(f"Produced: {flight}")
    time.sleep(3)


Produced: {'flight_id': 'FL1002', 'origin': (25.2048, 55.2708), 'destination': (1.3521, 103.8198), 'status': 'In Air', 'departure_time': 1759604440, 'arrival_time': 1759611482}
Produced: {'flight_id': 'FL1005', 'origin': (39.9042, 116.4074), 'destination': (19.076, 72.8777), 'status': 'Arrived', 'departure_time': 1759604440, 'arrival_time': 1759611345}
Produced: {'flight_id': 'FL1004', 'origin': (55.7558, 37.6173), 'destination': (41.9028, 12.4964), 'status': 'Delayed', 'departure_time': 1759604440, 'arrival_time': 1759611398}
Produced: {'flight_id': 'FL1007', 'origin': (51.5074, -0.1278), 'destination': (35.6895, 139.6917), 'status': 'Delayed', 'departure_time': 1759604440, 'arrival_time': 1759611172}
Produced: {'flight_id': 'FL1004', 'origin': (55.7558, 37.6173), 'destination': (41.9028, 12.4964), 'status': 'On Time', 'departure_time': 1759604440, 'arrival_time': 1759610836}
Produced: {'flight_id': 'FL1003', 'origin': (35.6895, 139.6917), 'destination': (37.7749, -122.4194), 'status'

KeyboardInterrupt: 